In [28]:
import langchain
print(langchain.__version__)

0.3.23


In [29]:
from langchain.agents import initialize_agent, Tool
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [30]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

agent_search_tool = Tool(
    name="Duck Duck go Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to search on web if answer is not present in Knowlwdge Base of LLM"
)

agent_llm = ChatGroq(
    model="llama3-70b-8192"
)

ddg_tool = [agent_search_tool]

search_agent = initialize_agent(ddg_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
agent_response = search_agent.run("What are the latest movies that will be release in April 2025")

print(agent_response)



> Entering new AgentExecutor chain...
Thought: I don't have the latest information about movie releases in April 2025, so I'll search online to find the answer.

Action: Duck Duck go Search
Action Input: "latest movies releasing in april 2025"
Thought: I don't have the latest information about movie releases in April 2025, so I'll search online to find the answer.

Action: Duck Duck go Search
Action Input: "latest movies releasing in april 2025"

Observation: If you're wondering what to watch the following month with so many new releases, we've got you covered. Below is a list of all the films that will be available in theaters in April 2025, with some ... April 2025 will bring a dose of horror with a vampire movie and a new take on Mickey Mouse, a movie based on a popular video game, and a lot more. Horror, fantasy, and a lot more. ... Published Mar 28, 2025. Follow Followed Like ... In conclusion, the April 2025 movie releases promise a vibrant cinematic experience packed with thri

In [31]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wikipedia_search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search on Wikipedia if answer is not present in Knowledge Base of LLM"
)
agent_llm = ChatGroq(
    model="llama3-70b-8192"
)
wiki_search_tool = [wikipedia_search_tool]
search_agent = initialize_agent(wiki_search_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
agent_response = search_agent.run("Give me the information about tesla car")
print(agent_response)



> Entering new AgentExecutor chain...
Thought: I don't have information about Tesla cars in my Knowledge Base, so I'll search on Wikipedia.

Action: Wikipedia Search
Action Input: "Tesla, Inc."Thought: I don't have information about Tesla cars in my Knowledge Base, so I'll search on Wikipedia.

Action: Wikipedia Search
Action Input: "Tesla, Inc."
Observation: Page: Tesla, Inc.
Summary: Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk led Tesla's first funding round and became the company's chairman; in 2008, he w

In [32]:
import sqlite3
from langchain import SQLDatabase
from langchain.agents import create_sql_agent

# Connect to the SQLite database
conn = sqlite3.connect("test_database.db")
cursor = conn.cursor()
# Create a sample table and insert some data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER NOT NULL,
        department TEXT NOT NULL
    )
''')
cursor.execute('''
    INSERT INTO employees (name, age, department)
    VALUES
        ('Alice', 30, 'HR'),
        ('Bob', 25, 'Engineering'),
        ('Charlie', 35, 'Sales')
''')
conn.commit()
conn.close()

print("Database created and sample data inserted.")

Database created and sample data inserted.


In [33]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

agent_llm = ChatGroq(
    model="llama3-70b-8192"
)

agent_db_onnect = SQLDatabase.from_uri("sqlite:///test_database.db")
agent_db_toolkit = SQLDatabaseToolkit(db=agent_db_onnect, llm=agent_llm)
agent_db_tools = agent_db_toolkit.get_tools()

sql_db_agent = initialize_agent(agent_db_tools, agent_llm, agent="zero-shot-react-description", verbose=True)

agent_response = sql_db_agent.run("What is the name of employee who is working in HR department")
print(agent_response)



> Entering new AgentExecutor chain...
Thought: I need to find the table that contains employee information and the department they are working in.

Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought:Thought: I need to find the table that contains employee information and the department they are working in.

Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought:Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	id INTEGER, 
	name TEXT NOT NULL, 
	age INTEGER NOT NULL, 
	department TEXT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	age	department
1	Alice	30	HR
2	Bob	25	Engineering
3	Charlie	35	Sales
*/
Thought:Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	id INTEGER, 
	name TEXT NOT NULL, 
	age INTEGER NOT NULL, 
	department TEXT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	age	department
1	Alice	30	HR
2	Bob	25	Engineer